In [1]:

import sys; sys.path.append('..')
import inflation, mesh, sheet_meshing
from tri_mesh_viewer import TriMeshViewer as Viewer
import triangulation
import numpy as np
import utils, py_newton_optimizer, benchmark
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import KDTree


In [2]:

V, E = mesh.load_raw('data/ForwardDesign/hinge.obj')
fusedPts = list(np.loadtxt('data/ForwardDesign/fusehingev3.txt').reshape((-1, 2)))
holePts = []
m, iwv, iwbv = sheet_meshing.forward_design_mesh(V, E, fusedPts, holePts, np.prod(utils.bbox_dims(V)[0:2]) / 1e4)
isheet = inflation.InflatableSheet(m, iwv)
viewer = Viewer(isheet, wireframe=True)
viewer.show()


Renderer(camera=PerspectiveCamera(children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0), quaternion=(…

In [3]:

target_points = np.array([
    [5.0,  0.0, 0.0],
    [5.0, 20.0, 0.0],
    [5.0, 40.0, 0.0]
])
V_rest = np.array([v.flatten() for v in isheet.restWallVertexPositions()])
tree = KDTree(V_rest)
tracked_indices = [tree.query(p)[1] for p in target_points]
trajectory = {idx: [] for idx in tracked_indices}
framerate = 5
def cb(it):
    if it % framerate == 0:
        for idx in tracked_indices:
            pos = isheet.getDeformedVtxPosition(idx, 0).flatten()
            trajectory[idx].append(pos)
        viewer.update()


In [4]:

benchmark.reset()
viewer.recordStart('tracked_inflation.mp4')
isheet.setUseTensionFieldEnergy(True)
isheet.setUseHessianProjectedEnergy(False)
opts = py_newton_optimizer.NewtonOptimizerOptions()
opts.niter = 500
isheet.pressure = 1
cr = inflation.inflation_newton(isheet, isheet.rigidMotionPinVars, opts, callback=cb)
benchmark.report()
viewer.recordStop()


AttributeError: 'TriMeshViewer' object has no attribute 'recordStart'

In [ ]:

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
for idx in tracked_indices:
    traj = np.array(trajectory[idx])
    if traj.shape[0] <= 1:
        continue
    ax.plot(traj[:, 0], traj[:, 1], traj[:, 2], label=f"Vertex {idx}")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title("3D Trajectory of Tracked Vertices During Inflation")
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:

fig, axs = plt.subplots(1, 3, figsize=(18, 5))
labels = [
    f"Vertex {idx} ({target_points[i][0]:.1f}, {target_points[i][1]:.1f}, {target_points[i][2]:.1f})"
    for i, idx in enumerate(tracked_indices)
]
for i, idx in enumerate(tracked_indices):
    traj = np.array(trajectory[idx])
    if traj.shape[0] <= 1:
        continue
    axs[0].plot(range(len(traj)), traj[:, 0], label=labels[i])
    axs[1].plot(range(len(traj)), traj[:, 1], label=labels[i])
    axs[2].plot(range(len(traj)), traj[:, 2], label=labels[i])
axs[0].set_title("X-axis Displacement")
axs[1].set_title("Y-axis Displacement")
axs[2].set_title("Z-axis Displacement")
for ax in axs:
    ax.set_xlabel("Iteration (every {} steps)".format(framerate))
    ax.set_ylabel("Displacement")
    ax.grid(True)
    ax.legend()
plt.tight_layout()
plt.show()


In [ ]:

def write_obj(path, V, F):
    with open(path, "w") as f:
        for v in V:
            f.write("v {:.6f} {:.6f} {:.6f}\n".format(*v))
        for face in F:
            f.write("f {} {} {}\n".format(*(face + 1)))

V_raw, F_raw, _, _ = isheet.visualizationGeometry()
V = np.array([v.flatten() for v in V_raw], dtype=np.float64)
F = np.array([list(f) for f in F_raw], dtype=np.int32)
write_obj("Tracked.obj", V, F)
